In [1]:
import mne
from neuro import *
from saflow_params import *
from behav import *
from utils import *
from scipy.io import loadmat

In [ ]:
PSD_data = load_PSD_data(FOLDERPATH, SUBJ_LIST, BLOCS_LIST, time_avg=True, stage='PSD')



In [23]:
PSD_allsubj, VTC_allsubj, subjIDX_allsubj = prepare_data(FOLDERPATH, LOGS_DIR, SUBJ_LIST, BLOCS_LIST)
print('PSD shape : {}'.format(PSD_allsubj.shape))
print('VTC shape : {}'.format(VTC_allsubj.shape))
print('subjIDX shape : {}'.format(subjIDX_allsubj.shape))

/home/karim/GitHub/saflow/scripts/neuro.py:211: RuntimeWarning: This filename (/storage/Yann/saflow_DATA/saflow_bids/sub-04/ses-recording/meg/sub-04_ses-recording_task-gradCPT_run-02_meg_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  events_artrej = mne.read_events(epo_filename, verbose=False) # get events from the epochs file (so no resp event)


KeyboardInterrupt: 

In [22]:
def prepare_data(FOLDERPATH, LOGS_DIR, SUBJ_LIST, BLOCS_LIST):
    PSD_allsubj = []
    VTC_allsubj = []
    subjIDX_allsubj = []
    for subjIDX, subj in enumerate(SUBJ_LIST):
        PSD_allblocs = []
        VTC_allblocs = []
        subjIDX_allblocs = []
        for bloc in BLOCS_LIST:
            INidx, OUTidx, VTC_epo, idx_trimmed = get_VTC_epochs(LOGS_DIR, subj, bloc, stage='epo', 
                                                                 lobound=None, hibound=None, save_epochs=False, 
                                                                 filt_order=3, filt_cutoff=0.1)
            PSD_fname, PSD_fpath = get_SAflow_bids(FOLDERPATH, subj, bloc, stage='PSD', cond=None)
            PSD = loadmat(PSD_fpath)['PSD']
            PSD_trimmed = np.array([PSD[:,:,:,idx] for idx in idx_trimmed]).swapaxes(0,3).swapaxes(2,1).swapaxes(0,2)
            PSD_trimmed = np.mean(PSD_trimmed, axis=2).squeeze()
            subjIDX_epo = np.ones(VTC_epo.shape)*subjIDX
            if len(PSD_allblocs)==0:
                PSD_allblocs = PSD_trimmed
                VTC_allblocs = VTC_epo
                subjIDX_allblocs = subjIDX_epo

            else:
                PSD_allblocs = np.concatenate((PSD_allblocs, PSD_trimmed), axis=2)
                VTC_allblocs = np.concatenate((VTC_allblocs, VTC_epo))
                subjIDX_allblocs = np.concatenate((subjIDX_allblocs, subjIDX_epo))

        if len(PSD_allsubj)==0:

            PSD_allsubj = PSD_allblocs
            VTC_allsubj = VTC_allblocs
            subjIDX_allsubj = subjIDX_allblocs
        else:
            PSD_allsubj = np.concatenate((PSD_allsubj, PSD_allblocs), axis=2)
            VTC_allsubj = np.concatenate((VTC_allsubj, VTC_allblocs))
            subjIDX_allsubj = np.concatenate((subjIDX_allsubj, subjIDX_allblocs))
    return PSD_allsubj, VTC_allsubj, subjIDX_allsubj

In [ ]:
# sub-06_ses-recording_task-gradCPT_run-04_meg_1600epo.fif.gz)

subj = '13'
bloc = 6

#for subj in SUBJ_LIST:
#    for bloc in BLOCS_LIST:
file = find_logfile(subj, bloc, os.listdir(LOGS_DIR))
print(file)
logfile = LOGS_DIR + '/' + file


print('Nombre de valeurs dans le respfile :')
data = loadmat(logfile)
df_response = pd.DataFrame(data['response'])
RT_array= np.asarray(df_response.loc[:,4])
print(RT_array.shape)


print('Nombre de valeurs dans la VTC : ')
VTC, INbounds, OUTbounds, INzone, OUTzone, RT_array = get_VTC_from_file(logfile, lobound=None, hibound=None, filt=True, filt_order=3, filt_cutoff=0.1)
print(VTC.shape)

print('Nombre d''events dans le fichier raw')
raw_fname, raw_fpath = get_SAflow_bids(FOLDERPATH, subj, bloc, stage='preproc_raw')
raw = mne.io.read_raw_fif(raw_fpath, preload=False, verbose=False)
try:
    events = mne.find_events(raw, min_duration=1/raw.info['sfreq'], verbose=False)
except ValueError:
    events = mne.find_events(raw, min_duration=2/raw.info['sfreq'], verbose=False)
count = 0
new_events = []
for ev in events:
    if ev[2] == 21 or ev[2] == 31:
        count = count +1
        new_events.append(ev)
print(count)


In [26]:
VTC_allsubj.shape


(37686,)

In [28]:
from scipy.io import savemat


mat = {'PSD_allsubj':PSD_allsubj,
      'VTC_allsubj':VTC_allsubj,
      'subjIDX_allsubj':subjIDX_allsubj}

savemat('/home/karim/pCloudDrive/science/saflow/features/PSD_VTCnosplit.mat', mat)